  
[**Dr. Kevin M. Moerman**](mailto:kevin.moerman@nuigalway.ie), _Lecturer in Biomedical Engineering_   

National University of Ireland Galway.   

---

This content is derived from a Jupyter notebook. Jupyter notebooks are a great way to combine teaching of theory and numerical implementation side-by-side and in an interactive manor.

* To run the Jupyter notebooks and to interact with the code uses should set-up Jupyter. Note that doing so is not a requirement for this course. All contant can be understood and studied without using the live code functionality. Students wishing to "skip" the codes can do so and focus on the theoretical content instead. 
* One way to set-up Jupyter is to install the Conda Python environment (https://www.anaconda.com/distribution/), which comes with Jupyter. See also instructions here: https://jupyter.org/
* For Octave (MATLAB syntax) notebooks users need to install Octave: https://www.gnu.org/software/octave/
* For Julia notebooks users could install Julia (https://julialang.org/) or use JuliaBox to run to notebooks directly in a browser (https://www.juliabox.com/). 
* The source for this notebook is found here: GitHub source: [https://github.com/Kevin-Mattheus-Moerman/NUIG_BME_402_6101](https://github.com/Kevin-Mattheus-Moerman/NUIG_BME_402_6101)


# The truss element
![spring](truss1.png)  

* Behaviour Hooke's law for a bar:    
$$\sigma=E\epsilon=\frac{F}{A}$$     
$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$   

* Truss (or bar) is a 1D element but has a **cross-sectional area** $A$    

* **strain** a relative metric for deformation: 
$$\epsilon=\frac{(u_2-u_1)}{L_0}$$    
  
$\sigma$: Stress (Pa or $N/m^2$)   
$F$: Force (N)   
$E$: The material Young's Modulus (Pa or $N/m^2$)   
$\epsilon$: The strain in the material ([])   
$L$ truss lenght (m)

## Example: A two truss system and a known force
Consider the three spring system below. 

* The system consists of 2 truss elements and 3 nodes.   

* Node 1 is contrained from moving.   

* A force of 10 N is applied to node 2 in the positive x-direction.   

$E^{(1)}=200\times10^3$ $MPa$   
$L^{(1)}=100$ $mm$   
$A^{(1)}=20$ $mm^2$   
   
$E^{(2)}=200\times10^3$ $MPa$   
$L^{(2)}=100$ $mm$  
$A^{(2)}=10$ $mm^2$   


![](trussSet_2_force.png)   

**Assignment**: Determine the nodal displacements, reaction forces and strains within each element of the bar.         


### Boundary and compatibility conditions
* The boundary conditions: 
$$u_1=0$$
$$F_3=P=10$$

### Constitutive behaviour
* Hooke's law for a truss   
    * Stress: 
    $$\sigma=E\epsilon=\frac{F}{A}$$
    * Strain: 
    $$\epsilon=\frac{(u_j-u_i)}{L_0}$$    

### The element equations
$$\begin{Bmatrix} F^{(i)} \end{Bmatrix}=\begin{bmatrix} K^{(i)}\end{bmatrix}\begin{Bmatrix} u^{(i)}\end{Bmatrix}$$  

$$\begin{Bmatrix} f_{1}^{(1)} \\ f_{2}^{(1)} \end{Bmatrix}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
  
$$\begin{Bmatrix} f_{2}^{(2)} \\ f_{3}^{(2)} \end{Bmatrix}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$


### Setting up the element stiffness matrices
* Recognize: 
$$K^{(i)}=\frac{A^{(i)}E^{(i)}}{L^{(i)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}=k^{(i)}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}$$   
* Derive individual stiffness values   
$$k^{(1)}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}=\frac{20*200\times10^3}{100}=40\times10^3$$
$$k^{(2)}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}=\frac{10*200\times10^3}{100}=20\times10^3$$
* Derive individual stiffness matrices   
$$K^{(1)}=\begin{bmatrix} 40 & -40 \\ -40 & 40\end{bmatrix}$$
$$K^{(2)}=\begin{bmatrix} 20 & -20 \\ -20 & 20\end{bmatrix}$$

In [2]:
I=[1 -1; -1 1]; # Array for spawning stiffness matrices
k1=(20*200)/100; #Stiffness 1
k2=(10*200)/100; #Stiffness 2
K1=k1*I; # Element stiffness matrix 1
K2=k2*I; # Element stiffness matrix 2
display(K1)
display(K2)

2×2 Array{Float64,2}:
  40.0  -40.0
 -40.0   40.0

2×2 Array{Float64,2}:
  20.0  -20.0
 -20.0   20.0

### Superposition to assemble global stiffness matrix
Through superposition to global stiffness matrix can be assembled. The superposition is often written as:
$$K=\sum_{i=1}^{N} K^{(i)}$$
However, it should be noted this is not a summation. The below numerical implementation illustrates how **the indices of the nodes involved are used as indices into matrix $K$**, leading to:
$$\begin{bmatrix} K \end{bmatrix}=\begin{bmatrix} 40 & -40 & 0 \\ -40 & 60 & -20 \\ 0 & -20 & 20\end{bmatrix}$$

In [10]:
K=zeros(3,3); #Initialize stiffness aray with all zeros
K[[1,2],[1,2]] .= K[[1,2],[1,2]] .+ K1 #Add element 1 contribution
K[[2,3],[2,3]] .= K[[2,3],[2,3]] .+ K2; #Add element 2 contribution
display(K)

3×3 Array{Float64,2}:
  40.0  -40.0    0.0
 -40.0   60.0  -20.0
   0.0  -20.0   20.0

### Solving the system using matrix algebra
* Following derivation of the global stiffness matrix, and using $u_1=0$, $F_2=0$, $F_3=P=10$, the total system now becomes:
$$\begin{Bmatrix} F_1 \\ 0 \\ 10 \end{Bmatrix}=\begin{bmatrix} 40 & -40 & 0 \\ -40 & 60 & -20 \\ 0 & -20 & 20\end{bmatrix}\begin{Bmatrix} 0 \\ u_2 \\ u_3\end{Bmatrix}$$
* Select the 2-3 sub-system: 
$$\begin{Bmatrix} 0 \\ 10 \end{Bmatrix}=\begin{bmatrix} 60 & -20 \\ -20 & 20\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$

* Solve: 
$$\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}=\begin{bmatrix} 60 & -20 \\ -20 & 20\end{bmatrix}^{-1}\begin{Bmatrix} 0 \\ 10 \end{Bmatrix}$$

In [22]:
F23=[0 10]'; #Force at node 2 and 3
K23=K[[2,3],[2,3]] #Stiffness matrix for 2-3 system
u23=K23\F23 #Displacement array for node 2 and 3, same as: inv(K23)*F23
display(u23)

2×1 Array{Float64,2}:
 0.25
 0.75

### Solving the system by solving system of linear equations manually
* Select the 2-3 sub-system: 
$$\begin{Bmatrix} 0 \\ 10 \end{Bmatrix}=\begin{bmatrix} 60 & -20 \\ -20 & 20\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$
* "Take out" two equations:
$$0=(60 u_2)+(-20 u_3)$$
$$10=(-20 u_2)+(20 u_3)$$
* Express $u_2$ in terms of $u_3$ or vice-versa:
$$0=(60 u_2)+(-20 u_3)\rightarrow 20 u_3=60 u_2 \rightarrow u_2=u_3/3  \rightarrow u_3=3u_2 $$
* Substitute into other equation e.g.:
$$10=(-20 [u_3/3])+(20 u_3) \rightarrow 10=(-20/3 u_3)+(20 u_3) \rightarrow 10=(20-20/3) u_3$$
$$\rightarrow u_3=\frac{10}{20-\frac{20}{3}}=0.75$$
$$\rightarrow u_2=u_3/3=0.25$$

### Compute force array F
Since all nodal diplacements are now known the full force array can now be computed from:
$$\begin{Bmatrix} F_1 \\ F_2 \\ F_3 \end{Bmatrix}=\begin{bmatrix} 40 & -40 & 0 \\ -40 & 60 & -20 \\ 0 & -20 & 20\end{bmatrix}\begin{Bmatrix} 0 \\ 0.25 \\ 0.75\end{Bmatrix}=\begin{Bmatrix} -10 \\ 0 \\ 10 \end{Bmatrix}$$

In [23]:
U=[0; u23[1]; u23[2]] #Full displacement array
F=K*U #Compute force array

3-element Array{Float64,1}:
 -10.0
   0.0
  10.0

### Computing element forces
The element force data can now be computed too from:
$$\begin{Bmatrix} F^{(i)} \end{Bmatrix}=\begin{bmatrix} k^{(i)}\end{bmatrix}\begin{Bmatrix} u^{(i)} \end{Bmatrix}$$   
  
E.g.:   

$$\begin{Bmatrix} f_{1x}^{(1)} \\ f_{2x}^{(1)} \end{Bmatrix}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}=\begin{bmatrix} k_1 & -k_1 \\ -k_1 & k_1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
$$\begin{Bmatrix} f_{2x}^{(2)} \\ f_{3x}^{(2)} \end{Bmatrix}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}=\begin{bmatrix} k_2 & -k_2 \\ -k_2 & k_2\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$

In [24]:
f1=K1*U[[1,2]]; #Element 1 forces
f2=K2*U[[2,3]]; #Element 2 forces
display(f1)
display(f2)

2-element Array{Float64,1}:
 -10.0
  10.0

2-element Array{Float64,1}:
 -10.0
  10.0

### Compute strains
* Hooke's law for a truss   
    * Stress: 
    $$\sigma=E\epsilon=\frac{F}{A}$$
    * Strain: 
    $$\epsilon=\frac{(u_j-u_i)}{L_0}$$    
    
$$\epsilon^{(1)}=\frac{(u_2-u_1)}{L^{(1)}}=\frac{(0.25-0)}{100}=0.0025$$    
$$\epsilon^{(2)}=\frac{(u_3-u_2)}{L^{(2)}}=\frac{(0.75-0.25)}{100}=0.005$$ 

## Example: A three spring system and a known force
Consider the three truss system below. 
* The system consists of 3 truss elements and 4 nodes.  

* Node 1, 3, and 4 are constrained from moving. 

* Truss 2 and 3 act in parallel and both originate from node 2  

* All truss lengths $L_i=1$  

* The truss Youngs Moduli are: $E_1=50$, $E_2=100$, and $E_3=25$  

* The truss cross sectional areas are: $A_1=10$, $A_2=10$, and $A_3=10$  

* A force of 25 kN is applied to node 2 in the positive x-direction.  

**Assignment**: Use the direct stiffness method to derive the finite element equations
![3 spring example](trussSet_3_force2.png)   

### Boundary and compatibility conditions
* The boundary conditions: 
$$u_1=u_3=u_4=0$$
* The compatibility condition:
$$u_2^{(1)}=u_2^{(2)}=u_2^{(3)}=u_2$$


### Setting up the element stiffness matrices
* Recognize: 
$$K^{(i)}=\frac{A^{(i)}E^{(i)}}{L^{(i)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}=k^{(i)}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}$$   
   
$$k^{(1)}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}=\frac{10*50}{1}=500$$
$$k^{(2)}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}=\frac{10*100}{1}=1000$$
$$k^{(3)}=\frac{A^{(3)}E^{(3)}}{L^{(3)}}=\frac{10*25}{1}=250$$

* Leading to:
$K^{(1)}=\begin{bmatrix} 500 & -500 \\ -500 & 500\end{bmatrix}$, $K^{(2)}=\begin{bmatrix} 1000 & -1000 \\ -1000 & 1000\end{bmatrix}$, and $K^{(3)}=\begin{bmatrix} 250 & -250 \\ -250 & 250\end{bmatrix}$

### Superposition to assemble global stiffness matrix
Through superposition to global stiffness matrix can be assembled. The superposition is often written as:
$$K=\sum_{i=1}^{3} K^{(i)}$$
However, it should be noted this is not a summation. The below numerical implementation illustrates how **the indices of the nodes involved are used as indices into matrix $K$**, leading to:
$$\begin{bmatrix} K \end{bmatrix}=\begin{bmatrix} 500 & -500 & 0 & 0 \\ -500 & 1750 & -1000 & -250 \\ 0 & -1000 & 1000 & 0 \\ 0 & -250 & 0 & 250 \end{bmatrix}$$

### Solving the system
* Following derivation of the global stiffness matrix, and using $u_1=u_3=u_4=0$, the total system now becomes:
$$\begin{Bmatrix} F_1 \\ 25000 \\ F_3 \\ F_4 \end{Bmatrix}=\begin{bmatrix} 500 & -500 & 0 & 0 \\ -500 & 1750 & -1000 & -250 \\ 0 & -1000 & 1000 & 0 \\ 0 & -250 & 0 & 250 \end{bmatrix}\begin{Bmatrix} 0 \\ u_2 \\ 0 \\ 0\end{Bmatrix}$$
* Leading to: 


$$25000=\begin{bmatrix}-500 & 1750 & -1000 & -250 \end{bmatrix} \begin{Bmatrix} 0 \\ u_2 \\ 0 \\ 0\end{Bmatrix}\rightarrow 25000=1750 u_2 \rightarrow u_2=\frac{25000}{1750}=14.2857...$$